In [58]:
import folium as fol
import pandas as pd

In [59]:
#Instantiate a map object with the Florence coordinates
m = fol.Map(location=[43.769871, 11.255576], zoom_start=13,tiles='cartodb positron')

In [60]:
# Example function to insert decimal point since coordinates
# are stored as integers in the Excel file
# (e.g. 4326 instead of 43.26)
def insert_decimal_point(s):
    return float(s[:2] + '.' + s[2:])

# Read the data
df = pd.read_excel('florence_venues.xlsx', sheet_name='Foglio2')

# Apply the conversion
df['latitude'] = df['latitude'].astype(str).apply(insert_decimal_point)
df['longitude'] = df['longitude'].astype(str).apply(insert_decimal_point)


In [61]:
df.columns

Index(['id', 'venue_name', 'venue_type', 'main_bus', 'venue_add_one',
       'venue_add_two', 'venue_pcode', 'venue_web', 'venue_phone',
       'venue_email', 'venue_fb', 'venue_tw', 'org_own', 'mger', 'mger_email',
       'mger_phone', 'online_contact_form', 'follow_up', 'venue_city',
       'venue_country', 'notes', 'std_sign_off', 'res_sign_off', 's_kick_id',
       'latitude', 'longitude', 'constituency', 'wday_opening', 'wday_closing',
       'wkd_opening', 'wkd_closing', 'show_opening', 'show_closing',
       'variations', 'promoters', 'venue_capacity', 'outdoor_space',
       'type_outdoor_space', 'Instagram', 'PRS Type', 'PRS Venue cat',
       'Year Venue Opened', 'Year Venue Closed', 'Notes'],
      dtype='object')

In [62]:
def check_missing(value, default="N/A"):
    """Return the value or a default string if the value is missing."""
    return value if pd.notna(value) else default

popup_template = """
<h4>{venue_name}</h4>
{address}
{postcode}
{links}
{phone}
{venue_type}
{capacity}
<b>Opening times:</b><br>
<i>Weekdays:</i> {weekdays}<br>
<i>Weekends:</i> {weekends}
"""

for index, row in df.iterrows():
    popup_text = popup_template.format(
        venue_name=check_missing(row['venue_name']),
        address=f"<b>Address:</b> {check_missing(row['venue_add_one'])} {check_missing(row['venue_add_two'])}<br>",
        postcode=f"<b>Postcode:</b> {check_missing(row['venue_pcode'])}<br>",
        links=f"<b>Links:</b> <a href='{check_missing(row['venue_web'])}'>Website</a> / <a href='{check_missing(row['venue_fb'])}'>Facebook</a> / <a href='{check_missing(row['Instagram'])}'>Instagram</a><br>",
        phone=f"<b>Phone:</b> {check_missing(row['venue_phone'])}<br>",
        venue_type=f"<b>Venue Type:</b> {check_missing(row['venue_type'])}<br>",
        capacity=f"<b>Capacity:</b> {check_missing(row['venue_capacity'])}<br>",
        weekdays=f"<b>Weekdays Opening:</b> {check_missing(row['wday_opening'])} - {check_missing(row['wday_closing'])}<br>",
        weekends=f"<b>Weekends Opening:</b> {check_missing(row['wkd_opening'])} - {check_missing(row['wkd_closing'])}<br>"
    )
    iframe = fol.IFrame(html=popup_text, width=250, height=250)
    fol.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['venue_name'],
        popup=fol.Popup(iframe, max_width=2650),
        icon=fol.Icon(color='red', icon='glyphicon glyphicon-music')
    ).add_to(m)


In [63]:
m

In [64]:
m.save('florence_venues.html')